In [0]:
%sql
drop database if exists dev.swatch_staging cascade;

In [0]:
%sql
drop table if exists dev.swatch_staging.workout_bpm;

In [0]:
class SetupSilverLayer:
    def __init__(self):
        self.catalog_name = "dev"
        self.silver_layer_schema_name = "swatch_staging"
        self.user_profile_table_name = "user_profile"
        self.complete_workout_table_name = "completed_workouts"
        self.workout_bpm_table_name = "workout_bpm"

In [0]:
class CreateSilverLayerTables(SetupSilverLayer):
    def __init__(self):
        super().__init__()
    
    def create_silver_layer_schema(self):
        spark.sql(f"create schema if not exists {self.catalog_name}.{self.silver_layer_schema_name}")
    
    def create_user_profile_table(self):
        spark.sql(f""" 
                  create table if not exists {self.catalog_name}.{self.silver_layer_schema_name}.{self.user_profile_table_name}
                  (
                    user_id string,
                    event_timestamp timestamp,
                    dob date,
                    sex string,
                    gender string,
                    first_name string,
                    last_name string,
                    street_address string, 
                    city string, 
                    state string, 
                    zip string
                  ) using delta
                  """)
        print(f"Table Created - {self.catalog_name}.{self.silver_layer_schema_name}.{self.user_profile_table_name}")

    def create_complete_workout_session_table(self):
        spark.sql(f""" 
                  create table if not exists {self.catalog_name}.{self.silver_layer_schema_name}.{self.complete_workout_table_name}
                  (
                      user_id string,
                      workout_id string,
                      session_id int,
                      start_time timestamp,
                      end_time timestamp
                  ) using delta
                  """)
        
        print(f"Table Created - {self.catalog_name}.{self.silver_layer_schema_name}.{self.complete_workout_table_name}")
    
    def create_workout_bpm_table(self):
        spark.sql(f""" 
                  create table  if not exists  {self.catalog_name}.{self.silver_layer_schema_name}.{self.workout_bpm_table_name}
                  (
                      user_id string,
                      device_id long,
                      workout_id string,
                      session_id int,
                      start_time timestamp,
                      end_time timestamp,
                      heartrate double
                  ) using delta
                  """)
        
        print(f"Table Created - {self.catalog_name}.{self.silver_layer_schema_name}.{self.workout_bpm_table_name}")
    
    def create_tables(self):
        self.create_silver_layer_schema()
        self.create_user_profile_table()
        self.create_complete_workout_session_table()
        self.create_workout_bpm_table()


In [0]:
class CreateSilverLayerCheckpoints(SetupSilverLayer):
    def __init__(self):
        super().__init__()
    
    def create_user_profile_checkpoint(self):
        spark.sql(f"create external volume if not exists {self.catalog_name}.{self.silver_layer_schema_name}.{self.user_profile_table_name}_checkpoint location 'abfss://smart-watch-analytics@dbstorageact.dfs.core.windows.net/swatch-checkpoint/staging_user_profile/' ")

        print(f"Checkpoint Created - {self.catalog_name}.{self.silver_layer_schema_name}.{self.user_profile_table_name}_checkpoint")
    
    def create_complete_workout_checkpoint(self):
        spark.sql(f"create external volume if not exists {self.catalog_name}.{self.silver_layer_schema_name}.{self.complete_workout_table_name}_checkpoint location 'abfss://smart-watch-analytics@dbstorageact.dfs.core.windows.net/swatch-checkpoint/staging_completed_workouts/' ")

        print(f"Checkpoint Created - {self.catalog_name}.{self.silver_layer_schema_name}.{self.complete_workout_table_name}_checkpoint")
        
    def create_workout_bpm_checkpoint(self):
        spark.sql(f"create external volume if not exists {self.catalog_name}.{self.silver_layer_schema_name}.{self.workout_bpm_table_name}_checkpoint location 'abfss://smart-watch-analytics@dbstorageact.dfs.core.windows.net/swatch-checkpoint/workout_bpm/' ")

        print(f"Checkpoint Created - {self.catalog_name}.{self.silver_layer_schema_name}.{self.workout_bpm_table_name}_checkpoint")

    def create_checkpoints(self):
        self.create_user_profile_checkpoint()
        self.create_complete_workout_checkpoint()
        self.create_workout_bpm_checkpoint()


In [0]:
silver_layer_tables = CreateSilverLayerTables()
silver_layer_tables.create_tables()

silver_layer_checkpoints = CreateSilverLayerCheckpoints()
silver_layer_checkpoints.create_checkpoints()


Table Created - dev.swatch_staging.user_profile
Table Created - dev.swatch_staging.completed_workouts
Table Created - dev.swatch_staging.workout_bpm
Checkpoint Created - dev.swatch_staging.user_profile_checkpoint
Checkpoint Created - dev.swatch_staging.completed_workouts_checkpoint
Checkpoint Created - dev.swatch_staging.workout_bpm_checkpoint
